In [ ]:
#1)
##Preprocessing titanic and adult data.
```{r}
adult=read.csv("C:\\Users\\student\\Documents\\Senior\\R\\adult.csv", header=FALSE)
names(adult)[1]="age"
names(adult)[2]="workclass"
names(adult)[3]="fnlwgt"
names(adult)[4]="education"
names(adult)[5]="educationnum"
names(adult)[6]="maritalstatus"
names(adult)[7]="occupation"
names(adult)[8]="relationship"
names(adult)[9]="race"
names(adult)[10]="sex"
names(adult)[11]="capitalgain"
names(adult)[12]="capitalloss"
names(adult)[13]="hoursperweek"
names(adult)[14]="nativecountry"
names(adult)[15]="target"
levels(adult$workclass)=c("Private", "Federal-gov", "Local-gov", "Never-worked", "Private", "Self-emp-inc", "Self-emp-not-inc", "State-gov", "Without-pay")
levels(adult$occupation)=c("Prof-specialty", "Adm-clerical", "Armed-Forces", "Craft-repair", "Exec-managerial", "Farming-fishing", "Handlers-cleaners", "Machine-op-inspct", "Other-service", "Priv-house-serv", "Prof-specialty", "Protective-serv", "Sales", "Tech-support", "Transport-moving")
levels(adult$nativecountry)=c("United-States", "Cambodia", "Canada", "China", "Columbia", "Cuba", "Dominican-Republic", "Ecuador", "El-Salvador", "England", "France", "Germany", "Greece", "Guatemala", "Haiti", "Holand-Netherlands", "Honduras", "Hong", "Hungary", "India", "Iran", "Ireland", "Italy", "Jamaica", "Japan", "Laos", "Mexico", "Nicaragua", "Outlying-US(Guam-USVI-etc)", "Peru", "Philippines", "Poland", "Portugal", "Puerto-Rico", "Scotland", "South", "Taiwan", "Thailand", "Trinadad&Tobago", "United-States", "Vietnam", "Yugoslavia")

titanic=read.csv("C:\\Users\\student\\Documents\\Senior\\R\\titanic.csv")
titanic$PassengerId =  NULL
titanic$Ticket =  NULL
titanic$Name = NULL
titanic$Cabin = NULL
titanic$Survived = factor(titanic$Survived)
titanic$Pclass = factor(titanic$Pclass)
names(titanic)[1]="target"
func94=function(x)
{
    for(j in 1:ncol(x))
    {
      for(i in 1:nrow(x))
      {
        if(is.numeric(x[,j])=="TRUE")
        {
          if(is.na(x[i,j])==1) x[i,j]=mean(x[,j], na.rm=TRUE)
          else x[i,j]=x[i,j]
        }
        else 
        {
          if(is.na(x[i,j])==1) x[i,j]=which.max(table(x[,j]))
          else x[i,j]=x[i,j]
        }
      }
    }
  return(x)
}
titanic<-func94(titanic)
```

#2)
##Train a random forest with 7-fold cross-validation.
###Adult

```{r}
library(caret)
model <- train(target~.,data = adult, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE))
cm=confusionMatrix(data = model, reference = adult$target, positive = "2")
cm
```

###Accuracy=.8569

###Titanic

```{r}
model2 <- train(target~.,data = titanic, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE))
cm2=confusionMatrix(data = model2, reference = titanic$target, positive = "1")
cm2
```

###Accuracy=.8215

#3)
##Train a random forest with 10-fold cross-validation.
###Adult

```{r}
model3 <- train(target~.,data = adult, method = "ranger", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))
cm3=confusionMatrix(data = model3, reference = adult$target, positive = "2")
cm3
```

###Accuracy=.8577

###Titanic

```{r}
model4 <- train(target~.,data = titanic, method = "ranger", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))
cm4=confusionMatrix(data = model4, reference = titanic$target, positive = "1")
cm4
```

###Accuracy=.8215

#4)
##Reporting hyperparameters from #2)
###Adult: mtry=49, splitrule=gini, min.node.size=1
###Titanic: mtry=2, splitrule=gini, min.node.size=1

##Reporting hyperparameters from #3)
###Adult: mtry=49, splitrule=gini, min.node.size=1
###Titanic: mtry=6, splitrule=gini, min.node.size=1

#5)
##Print the plot of forests from #2)
###Adult

```{r}
plot(model)
```

###Titanic

```{r}
plot(model2)
```

##Print the plot of forests from #3)
###Adult

```{r}
plot(model3)
```

###Titanic

```{r}
plot(model4)
```

#6)
##Tune the three hyperparamets.
###Adult
###You gave me permission to skip this part due to the time it would take to complete it.
               
###Titanic

```{r}
myGrid2 = expand.grid(mtry = c(1:(ncol(titanic)-1)), splitrule = c("gini","extratrees"),
                     min.node.size = c(1:20))
model6 <- train(target~.,data = titanic, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid2)
```

###mtry=7, splitrule=gini, min.node.size=20

#7) 
##Printing plot and reporting accuracy from #6) in comparison to #2).
###Adult: 
###I skipped the adult piece for #6), so I have no accuracy to report here.

###Titanic

```{r}
print(model6)
plot(model6)
cm6=confusionMatrix(data = model6, reference = titanic$target, positive = "1")
cm6
```

###Accuracy=.8485
###The tuned forest does achieve better accuracy than the default forest in #2.

#8)
##Using default and custom glmnet.
###Adult
###Default

```{r}
levels(adult$target)=c("UnderEqual", "Over")
mycontrol1<-trainControl(method="cv", number=7, summaryFunction=twoClassSummary, classProbs=TRUE, verboseIter=TRUE)
model7<-train(target~., adult, method="glmnet", trControl=mycontrol1)
model7
plot(model7)
cm7=confusionMatrix(data = model7, reference = adult$target, positive = "2")
cm7
```

###Accuracy=.8505

###Custom

```{r}
myGrid3<-expand.grid(alpha=0:1, lambda=seq(.0001, 1, length=100))
model8<-train(target~., adult, method="glmnet", tuneGrid=myGrid3, trControl=mycontrol1)
model8
plot(model8)
cm8=confusionMatrix(data = model8, reference = adult$target, positive = "2")
cm8
```

###Accuracy=.8506
###The tuned glmnet model had a slightly better performance than the default glmnet model.

###Titanic
###Default

```{r}
levels(titanic$target)=c("NotSurvived", "Survived")
mycontrol2<-trainControl(method="cv", number=7, summaryFunction=twoClassSummary, classProbs=TRUE, verboseIter=TRUE)
model9<-train(target~., titanic, method="glmnet", trControl=mycontrol2)
model9
plot(model9)
cm9=confusionMatrix(data = model9, reference = titanic$target, positive = "2")
cm9
```

###Accuracy=.8013

###Custom Tuning

```{r}
myGrid4<-expand.grid(alpha=0:1, lambda=seq(.0001, 1, length=100))
model10<-train(target~., titanic, method="glmnet", tuneGrid=myGrid4, trControl=mycontrol2)
model10
plot(model10)
cm10=confusionMatrix(data = model10, reference = titanic$target, positive = "2")
cm10
```

###Accuracy=.8025
###The tuned glmnet model had a slightly better performance than the default glmnet model.